When learning games, humans need experience. To become an expert we need lots of experience. First we must learn what the game's environment looks like, and how we can interact with said environment. Most importantly though, we learn how to win by studying past wins and failures.

Computers fortunately can learn in a similar fashion. The environment and agent relationship must be defined, which is generally straightforward, and often tedious. The magic happens when we are able to calculate a value function and update each state's value. This is demonstrated with a simple reinforcement learning algorithm that can be applied for learning tic-tac-toe.

Just use the update rule where V(s) is the value of the previous state, V(s') is the value of the next state, and alpha is the learning rate: 

V(s) = V(s) + alpha * (V(s') - V(s))

Alpha is the hyperparameter that allows us to control how important new info is to updating our value function. If alpha is closer to 1 in this case then the value of the new state will override past info more easily. If its closer to 0 then the update of the new state is less influential. This can be good for when the agent is exploring new options in its environment, which brings us to the second hyperparameter we have control over in this situation, epsilon.

In laymans terms, epsilon is the rate at which our agent explores its environment. Explore is this case means it will not choose the option that it has learned is most beneficial, but will explore a new action to see if that action is better than what it had previously learned. 

So how do the two work together? Well if you have a high learning rate and high epsilon, then your agent will explore often and use those new random explorations' values to "override" previous states' values, which would make the agent jump to conclusions over and over. We would rather have an agent use more careful learning methods and explore less, so having a lower alpha and epsilon would be smart (under 0.5). 

But, why take my word for it? Let's try out a few different hyperparameter combonations.

Interesting things to try:
1. What if they have different learning rates?
2. What if they have different epsilons? (probability of exploring)
3. What if the agent doesn't have enough experience?

In [1]:
# Note: you may need to update your version of future: sudo pip install -U future
from __future__ import print_function, division
from builtins import range, input

import numpy as np
import matplotlib.pyplot as plt

LENGTH = 3

class Agent:
    def __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps # probability of choosing random action 
        self.alpha = alpha # learning rate
        self.verbose = False
        self.state_history = []

    def setV(self, V):
        self.V = V
    
    # Assigns symbol to Agent, whether that be 1 (O) or -1 (X)
    def set_symbol(self, sym):
        self.sym = sym
        
    # If true, will print values for each position on the board
    def set_verbose(self, v):
        self.verbose = v
    
    # Erases all historical states into empty list
    def reset_history(self):
        self.state_history = []
        
    # Choose an action based on epsilon-greedy strategy
    def take_action(self, env):
        r = np.random.rand()
        best_state = None
        # If random number from 0 to 1 is less than our epsilon value then take a random action
        if r < self.eps:
            if self.verbose:
                print("Taking a random action")
            # Look at all possible moves by remembering which of the positions are open...
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            # ...and randomly choosing one of those positions
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
        else:
            # Choose the best action based on current values of states
            # Loop through all possible moves, get their values and keep track of the best value
            pos2value = {} # for debugging
            next_move = None
            best_value = -1
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                    # what is the state if we made this move?
                        env.board[i,j] = self.sym
                        state = env.get_state()
                        env.board[i,j] = 0 # don't forget to change it back!
                        pos2value[(i,j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)

            # if verbose, draw the board w/ the values
            if self.verbose:
                print("Taking a greedy action")
                for i in range(LENGTH):
                    print("------------------")
                    for j in range(LENGTH):
                        if env.is_empty(i, j):
                            # print the value
                            print(" %.2f|" % pos2value[(i,j)], end="")
                        else:
                            print("  ", end="")
                            if env.board[i,j] == env.x:
                                print("x  |", end="")
                            elif env.board[i,j] == env.o:
                                print("o  |", end="")
                            else:
                                print("   |", end="")
                    print("")
                print("------------------")

        # make the move
        env.board[next_move[0], next_move[1]] = self.sym

    def update_state_history(self, s):
        # Not in take_action, because take_action happens for each player
        # State history needs to be updated every iteration
        # s = env.get_state() # don't want to do this twice so pass it in
        self.state_history.append(s)

    def update(self, env):
        # We want to BACKTRACK over the states, so that:
        # V(prev_state) = V(prev_state) + alpha*(V(next_state) - V(prev_state)) where V(next_state) = reward if it's the most current state
        # NOTE: we ONLY do this at the end of an episode
        reward = env.reward(self.sym)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha*(target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()


In [2]:
# This class represents a tic-tac-toe game environment
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1 # represents an x on the board, player 1
        self.o = 1 # represents an o on the board, player 2
        self.winner = None
        self.ended = False
        self.num_states = 3**(LENGTH*LENGTH)

    def is_empty(self, i, j):
        return self.board[i,j] == 0

    def reward(self, sym):
        # No reward until game is over
        if not self.game_over():
            return 0
        # If we get here, game is over
        # sym will be self.x or self.o
        return 1 if self.winner == sym else 0

    def get_state(self):
        # Returns the current state, represented as an int from 0...|S|-1, where S = set of all possible states
        # |S| = 3^(BOARD SIZE), since each cell can have 3 possible values - empty, x, o
        # Some states are not possible, e.g. all cells are x, but we ignore that detail
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i,j] == 0:
                    v = 0
                elif self.board[i,j] == self.x:
                    v = 1
                elif self.board[i,j] == self.o:
                    v = 2
                h += (3**k) * v
                k += 1
        return h

    def game_over(self, force_recalculate=False):
        # Returns true if game over (a player has won or it's a draw) otherwise returns false
        # Also sets 'winner' instance variable and 'ended' instance variable
        if not force_recalculate and self.ended:
            return self.ended

        # Check rows
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        # Check columns
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:,j].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        # Check diagonals
        for player in (self.x, self.o):
            # top-left -> bottom-right diagonal
            if self.board.trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
            # top-right -> bottom-left diagonal
            if np.fliplr(self.board).trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True

        # Check if draw
        if np.all((self.board == 0) == False):
            # winner stays None
            self.winner = None
            self.ended = True
            return True

        # Game is not over
        self.winner = None
        return False

    def is_draw(self):
        return self.ended and self.winner is None

    # Example board
    # -------------
    # | x |   |   |
    # -------------
    # |   |   |   |
    # -------------
    # |   |   | o |
    # -------------
    def draw_board(self):
        for i in range(LENGTH):
            print("-------------")
            for j in range(LENGTH):
                print("  ", end="")
                if self.board[i,j] == self.x:
                    print("x ", end="")
                elif self.board[i,j] == self.o:
                    print("o ", end="")
                else:
                    print("  ", end="")
            print("")
        print("-------------")

In [3]:
class Human:
    def __init__(self):
        pass

    def set_symbol(self, sym):
        self.sym = sym

    def take_action(self, env):
        while True:
        # break if we make a legal move
            move = input("Enter coordinates i,j for your next move (i,j=0..2): ")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i,j] = self.sym
                break

    def update(self, env):
        pass

    def update_state_history(self, s):
        pass

In [4]:
# Recursive function that will return all possible states (as ints) and who the corresponding winner is for those states (if any)
# (i, j) refers to the next cell on the board to permute (we need to try -1, 0, 1)
# Impossible games are ignored, i.e. 3x's and 3o's in a row simultaneously since that will never happen in a real game
def get_state_hash_and_winner(env, i=0, j=0):
    results = []
    for v in (0, env.x, env.o):
        env.board[i,j] = v # if empty board it should already be 0
        if j == 2:
            # j goes back to 0, increase i, unless i = 2, then we are done
            if i == 2:
                # The board is full, collect results and return
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
            else:
                results += get_state_hash_and_winner(env, i + 1, 0)
        else:
        # Increment j, i stays the same
            results += get_state_hash_and_winner(env, i, j + 1)

    return results

def initialV_x(env, state_winner_triples):
    # Initialize state values as follows:
    # If x wins, V(s) = 1
    # If x loses or draw, V(s) = 0
    # Otherwise, V(s) = 0.5
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V


def initialV_o(env, state_winner_triples):
    # This is (almost) the opposite of initial V for player x since everywhere where x wins (1), o loses (0)
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V


def play_game(p1, p2, env, draw=False):
    # Loops until the game is over
    current_player = None
    while not env.game_over():
        # Alternate between players
        # p1 always starts first
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1

        # Draw the board before the user who wants to see it makes a move
        if draw:
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()

        # Current player makes a move
        current_player.take_action(env)

        # Update state histories
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)

    if draw:
        env.draw_board()

    # Do the value function update
    p1.update(env)
    p2.update(env)

In [5]:
if __name__ == '__main__':
    # Train the agents
    p1 = Agent(eps = 0.1, alpha = 0.5)  # This is the baseline
    p2 = Agent(eps = 0.9, alpha = 0.5)  # High epsilson
    p3 = Agent(eps = 0.1, alpha = 0.01) # Low alpha and few training rounds
    p4 = Agent(eps = 0.1, alpha = 0.01) # Low alpha and many training rounds
    p5 = Agent(eps = 0.1, alpha = 0.9)  # High alpha and few training rounds
    p6 = Agent(eps = 0.1, alpha = 0.9)  # High alpha and many training rounds

    # Set initial V for p1 and p2
    env = Environment()
    state_winner_triples = get_state_hash_and_winner(env)

    Vx = initialV_x(env, state_winner_triples)
    Vo = initialV_o(env, state_winner_triples)
    
    p1.setV(Vo) # p1 is always 'o'
    p2.setV(Vx) 
    p3.setV(Vx)
    p4.setV(Vx)
    p5.setV(Vx)
    p6.setV(Vx)

    # Give each player their symbol
    p1.set_symbol(env.o) # p1 is always 'o'
    p2.set_symbol(env.x)
    p3.set_symbol(env.x)
    p4.set_symbol(env.x)
    p5.set_symbol(env.x)
    p6.set_symbol(env.x)
    
    high_exp = 1000
    low_exp = 10
    
    print("Train p2")    
    for t in range(high_exp):
        if t % 200 == 0:
            print(t)
        play_game(p2, p1, Environment())
        
    print("Train p3")        
    for t in range(low_exp):
        if t % 20 == 0:
            print(t)
        play_game(p3, p1, Environment())  
        
    print("Train p4")
    for t in range(high_exp):
        if t % 200 == 0:
            print(t)
        play_game(p4, p1, Environment())
        
    print("Train p5")    
    for t in range(low_exp):
        if t % 2 == 0:
            print(t)
        play_game(p5, p1, Environment())  
        
    print("Train p6")    
    for t in range(high_exp):
        if t % 200 == 0:
            print(t)
        play_game(p6, p1, Environment()) 

Train p2
0
200
400
600
800
Train p3
0
Train p4
0
200
400
600
800
Train p5
0
2
4
6
8
Train p6
0
200
400
600
800


Since I'm just starting off I want to give myself a chance against this first agent. p1 has been trained as a baseline opponent against all the other agents, so it has plenty of experience. Do note though, p1 has consistently been trained as the second player, so it knows how to act in response to another player starting the game. Let's see if I can beat it given I have the upper hand in going first.

In [6]:
human = Human()
human.set_symbol(env.x) # Only this time my symbol is 'x' because p1 was 'o' while training the other agents.

# Human vs. player 1 (eps = 0.1,  alpha = 0.5). Human goes first
while True:
    p1.set_verbose(True)
    play_game(human, p1, Environment(), draw=1)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

-------------
            
-------------
            
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 1,1
Taking a greedy action
------------------
 0.00| 0.00| 0.00|
------------------
 0.00|  x  | 0.00|
------------------
 0.00| 0.18| 0.00|
------------------
-------------
            
-------------
      x     
-------------
      o     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,2
Taking a greedy action
------------------
 0.00| 0.00|  x  |
------------------
 0.00|  x  | 0.00|
------------------
 0.25|  o  | 0.00|
------------------
-------------
          x 
-------------
      x     
-------------
  o   o     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,2
Taking a greedy action
------------------
 0.00| 0.00|  x  |
------------------
 0.00|  x  | 0.00|
------------------
  o  |  o  |  x  |
------------------
-------------
          x 
-------------
      x   o 
-------------
  o   o 

Phew, I did it. It knew to block me but since I started I used a strategy to outsmart it. Let's see how well I can do with the other agents. These times they will be starting the game (since draw = 2 in play game). The good thing for me is that p2 will randomly explore 90% of the time, so even with it starting our game, I am at a huge advantage.

In [7]:
human = Human()
human.set_symbol(env.o) # I will now be 'o' since they were trained as 'x'

# Human vs. player 2 (eps = 0.9,  alpha = 0.5). Player 2 goes first
while True:
    p2.set_verbose(True)
    play_game(p2, human, Environment(), draw=2)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

Taking a random action
-------------
            
-------------
          x 
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 1,1
Taking a random action
-------------
  x         
-------------
      o   x 
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,1
Taking a greedy action
------------------
  x  | 0.25| 0.00|
------------------
 0.20|  o  |  x  |
------------------
 0.00|  o  | 0.00|
------------------
-------------
  x   x     
-------------
      o   x 
-------------
      o     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,2
Taking a random action
-------------
  x   x   o 
-------------
  x   o   x 
-------------
      o     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,0
-------------
  x   x   o 
-------------
  x   o   x 
-------------
  o   o     
-------------
Play again? [Y/n]: n


Fortunately for me p2 had a 90% chance of making a random decision which hurt its decision making. I was quickly able to get the best of it since it did not exploit it's stragegy, but was looking for another solution. Now for something a bit more challenging. p3 has a low epsilon, which means it will explore less and exploit more. It also will only update incrementally so that it will slowly converge on an optimal strategy. This one however, will only have played 10 games. Let's see if it learned enough in that time.

In [10]:
human = Human()
human.set_symbol(env.o)

# Human vs. player 3 (eps = 0.1,  alpha = 0.01) with 10 training rounds. Player 3 goes first
while True:
    p3.set_verbose(True)
    play_game(p3, human, Environment(), draw=2)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

Taking a greedy action
------------------
 0.12| 0.10| 0.32|
------------------
 0.17| 0.99| 0.03|
------------------
 0.11| 0.26| 0.29|
------------------
-------------
            
-------------
      x     
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,2
Taking a greedy action
------------------
 0.50| 0.54| 0.19|
------------------
 0.91|  x  | 0.93|
------------------
 0.76| 0.76|  o  |
------------------
-------------
            
-------------
      x   x 
-------------
          o 
-------------
Enter coordinates i,j for your next move (i,j=0..2): 1,0
Taking a greedy action
------------------
 0.17| 0.50| 0.14|
------------------
  o  |  x  |  x  |
------------------
 0.50| 0.95|  o  |
------------------
-------------
            
-------------
  o   x   x 
-------------
      x   o 
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,1
Taking a greedy action
------------------
 0.04|  o  | 0.14|
---------------

Alright, now we are getting somewhere. With only 10 rounds it actually learned some decent strategy. The very low epsilon gives such a low chance for exploration that all the actions taken were "greedy" or in other words, the maximized the chance of reward. This is also known as exploitation.

Now I'll be doing the same looking agent but with many more rounds of training to see if more experience pays off in this scenario.

In [6]:
human = Human()
human.set_symbol(env.o)

# Human vs. player 4 (eps = 0.1,  alpha = 0.01) with 1000 rounds of training. Player 4 goes first
while True:
    p4.set_verbose(True)
    play_game(p4, human, Environment(), draw=2)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

Taking a greedy action
------------------
 0.33| 0.04| 0.05|
------------------
 0.03| 0.68| 0.18|
------------------
 0.30| 0.79| 0.04|
------------------
-------------
            
-------------
            
-------------
      x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 1,1
Taking a greedy action
------------------
 0.03| 0.04| 0.01|
------------------
 0.01|  o  | 0.03|
------------------
 0.02|  x  | 0.03|
------------------
-------------
      x     
-------------
      o     
-------------
      x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,0
Taking a greedy action
------------------
  o  |  x  | 0.01|
------------------
 0.01|  o  | 0.06|
------------------
 0.50|  x  | 0.13|
------------------
-------------
  o   x     
-------------
      o     
-------------
  x   x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,2
-------------
  o   x     
-------------
      o     
-------------
  x   x 

How strange! With more training this agent performed much worse. I am honstly shocked at how poorly it performed given how much it trained. It must have memorized sequences of moves, but not learned general stragety. Perhaps it fell into a local optimum set of decisions based off who it was trained against (p1), but when pitted against a human, did not know what to do. 

Now let's see how a high learning rate agent does with only 10 rounds of training.

In [7]:
human = Human()
human.set_symbol(env.o)

# Human vs. player 5 (eps = 0.1,  alpha = 0.9) with 10 rounds of training. Player 5 goes first
while True:
    p5.set_verbose(True)
    play_game(p5, human, Environment(), draw=2)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

Taking a greedy action
------------------
 0.33| 0.04| 0.05|
------------------
 0.03| 0.68| 0.18|
------------------
 0.30| 0.79| 0.04|
------------------
-------------
            
-------------
            
-------------
      x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 1,1
Taking a greedy action
------------------
 0.03| 0.04| 0.01|
------------------
 0.01|  o  | 0.03|
------------------
 0.02|  x  | 0.03|
------------------
-------------
      x     
-------------
      o     
-------------
      x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,2
Taking a greedy action
------------------
 0.01|  x  |  o  |
------------------
 0.01|  o  | 0.12|
------------------
 0.17|  x  | 0.03|
------------------
-------------
      x   o 
-------------
      o     
-------------
  x   x     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,2
Taking a greedy action
------------------
 0.05|  x  |  o  |
---------------

p5 behaved similarly to the p4. Its as if they tried going for the win without considering my moves or trying to block me. This case makes more sense to me however, since its learning rate was so high. It was basically going off its last game more than its prior game(s) which made it very biased toward the final training game it went through.

In [8]:
human = Human()
human.set_symbol(env.o)

# Human vs. player 5 (eps = 0.1,  alpha = 0.9) with 1000 rounds of training. Player 5 goes first
while True:
    p6.set_verbose(True)
    play_game(p6, human, Environment(), draw=2)
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
        break

Taking a greedy action
------------------
 0.33| 0.04| 0.05|
------------------
 0.03| 0.68| 0.18|
------------------
 0.30| 0.14| 0.04|
------------------
-------------
            
-------------
      x     
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,0
Taking a greedy action
------------------
  o  | 0.99| 0.13|
------------------
 0.17|  x  | 0.15|
------------------
 0.03| 0.08| 0.03|
------------------
-------------
  o   x     
-------------
      x     
-------------
            
-------------
Enter coordinates i,j for your next move (i,j=0..2): 2,1
Taking a random action
-------------
  o   x     
-------------
      x     
-------------
  x   o     
-------------
Enter coordinates i,j for your next move (i,j=0..2): 0,2
Taking a greedy action
------------------
  o  |  x  |  o  |
------------------
 0.09|  x  | 0.06|
------------------
  x  |  o  | 0.01|
------------------
-------------
  o   x   o 
-------------
  x   x     

p6 actually seemed quite smart. Though it was relying more on its later training games, it must have gone through so many that the learning rate was not as critical at some point. 




Summary:

There are a limited number of actions and states so tic-tac-toe may be too simple to really tweak the reinforcement learning hyperparameters to get super varied results, but this does give a view into the control one may have when training these agents. However, it looks as though agents/players have very dramatic performance differences when epsilon is shifted between 0 and 1. This makes sense, since epsilon is the cutoff that determines the "chance" of the agent choosing a random action to explore other states. When playing an actual player, any chance of random play could cost you the game.

Hopefully this can help you get a feel for reinforcement learning on a simple game and will allow you to try to take the next step towards bigger games, and eventually into practical applications. Good luck learning!

Sources:

https://deeplearningcourses.com/c/artificial-intelligence-reinforcement-learning-in-python

https://www.udemy.com/artificial-intelligence-reinforcement-learning-in-python